- Import Packages

In [1]:
import os
import random
import math
import json
import pybrain
import pandas

- Take out "class" and "content_cut"

In [60]:
for i,j,k in os.walk('../event_content_cut_json'):
    file_names = k

events = []
for file_name in file_names:
    class_name = file_name[:-4]
    print(file_name)
    for line in open('../event_content_cut_json/'+file_name,'r').readlines():
        event = {}        
        keywords = line[16:-4].split(',')
        event['class'] = class_name
        event['content_cut'] = keywords
        events.append(event)
json.dump(events,open('events.json','w'))

arts.txt
business.txt
charity.txt
education.txt
entertainment.txt
fashion.txt
food.txt
health.txt
other.txt
photography.txt
sports.txt
technology.txt
travel.txt


- Sampling (200 samples for each class)

In [35]:
events = json.load(open('events.json','r'))
all_data_index = [i for i in range(40509)]
training_data_index = []
count_dict = {}
count = 0
all_sample_index = [i for i in range(40509)]
while True:    
    n = random.choice(all_sample_index)
    all_sample_index.remove(n)
    event = events[n]
    if event['class'] not in count_dict:
        training_data_index.append(n)
        count_dict[event['class']] = 1
        count += 1
    elif count_dict[event['class']] < 200:
        training_data_index.append(n)
        count_dict[event['class']] += 1
        count += 1
    else:
        continue
    if count == 2600:
        break

testing_data_index = list(set(all_data_index)-set(training_data_index))

training_data = []
for index in training_data_index:
    training_data.append(events[index])
testing_data = []
for index in testing_data_index:
    testing_data.append(events[index])
json.dump(training_data,open('training_data.json','w'))
json.dump(testing_data,open('testing_data.json','w'))

json.dump(training_data,open('training_data.json','w'))
json.dump(testing_data,open('testing_data.json','w'))

- Merge "content_cut" for each class

In [37]:
training_data = json.load(open('training_data.json','r'))
testing_data = json.load(open('testing_data.json','r'))

class_content_dict = {"arts":[],
                      "business":[],
                      "charity":[],
                      "education":[],
                      "entertainment":[],
                      "fashion":[],
                      "food":[],
                      "health":[],
                      "other":[],
                      "photography":[],
                      "sports":[],
                      "technology":[],
                      "travel":[]}
for data in training_data:
    class_content_dict[data['class']].extend(data['content_cut'])


json.dump(class_content_dict,open('class_content_dict.json','w'))

- Remove numbers and alphabets

In [38]:
class_content_dict = json.load(open('class_content_dict.json','r'))

class_content_dict_2 = {"arts":[],
                      "business":[],
                      "charity":[],
                      "education":[],
                      "entertainment":[],
                      "fashion":[],
                      "food":[],
                      "health":[],
                      "other":[],
                      "photography":[],
                      "sports":[],
                      "technology":[],
                      "travel":[]}

for class_name, content in class_content_dict.items():
    for keyword in content:
        flag = True
        for word in keyword:
            char = word.encode('unicode_escape').decode('utf8')
            if not (char>='\\u4e00' and char<='\\u9fa5'):
                flag = False
                break
        if flag:
            class_content_dict_2[class_name].append(keyword)


json.dump(class_content_dict_2,open('class_content_dict_2.json','w'))

- Calculate TF/IDF

In [47]:
class_content_dict = json.load(open('class_content_dict_2.json','r'))

keyword_count_dict = {}
keyword_freq_dict = {}
for class_name, keywords in class_content_dict.items():  
    print(class_name)
    keyword_count_dict[class_name] = {}
    for keyword in keywords:
        if keyword in keyword_count_dict[class_name]:
            keyword_count_dict[class_name][keyword] += 1 
        else:
            keyword_count_dict[class_name][keyword] = 1 
            if keyword in keyword_freq_dict:
                keyword_freq_dict[keyword] += 1
            else:
                keyword_freq_dict[keyword] = 1

keyword_tf_dict = {}
keyword_idf_dict = {}
for class_name, keywords in class_content_dict.items():  
    length = len(keywords)
    keyword_tf_dict[class_name] = {}
    for keyword, count in keyword_count_dict[class_name].items():
        keyword_tf_dict[class_name][keyword] = count/length
for keyword,freq in keyword_freq_dict.items():
    keyword_idf_dict[keyword] = math.log10(13/freq)

keyword_tfidf_dict = {}
for class_name, keyword_tf in keyword_tf_dict.items():
    keyword_tfidf_dict[class_name] = {}
    for keyword, tf in keyword_tf.items():
        keyword_tfidf_dict[class_name][keyword] = keyword_tf_dict[class_name][keyword]*keyword_idf_dict[keyword]

json.dump(keyword_tfidf_dict,open('keyword_tfidf_dict.json','w'))

fashion
technology
sports
health
charity
arts
food
other
entertainment
photography
education
travel
business


- Select input

In [97]:
keyword_tfidf_dict = json.load(open('keyword_tfidf_dict.json','r'))


input_list = []
for class_name, keyword_tfidf in keyword_tfidf_dict.items():
    for keyword, tfidf in keyword_tfidf.items():
        if tfidf > 1e-4:
            input_list.append(keyword)

input_list = list(set(input_list))

print(len(input_list))
print(int(math.sqrt(len(input_list)*13)))

1716
149


- Build input

In [99]:
training_data = json.load(open('training_data.json','r'))
testing_data = json.load(open('testing_data.json','r'))

count = 0
for data in training_data:
    count += 1
    if count % 1000 == 0:
        print(count)
    data['input'] = []
    for keyword in input_list:
        if keyword in data['content_cut']:
            data['input'].append(data['content_cut'].count(keyword))
        else:
            data['input'].append(0)
count = 0
for data in testing_data:
    count += 1
    if count % 1000 == 0:
        print(count)
    data['input'] = []
    for keyword in input_list:
        if keyword in data['content_cut']:
            data['input'].append(data['content_cut'].count(keyword))
        else:
            data['input'].append(0)

1000
2000
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000


In [55]:
from pybrain.datasets import ClassificationDataSet
from pybrain.supervised.trainers import BackpropTrainer
from pybrain.tools.shortcuts import buildNetwork

In [100]:
class_names = []
for class_name in iter(keyword_tfidf_dict):
    class_names.append(class_name)

DS = ClassificationDataSet(len(input_list), nb_classes = 13)
for data in training_data:
    DS.appendLinked(data['input'],class_names.index(data['class']))
    
DS._convertToOneOfMany(bounds=[0, 1])

In [101]:
net = buildNetwork(len(input_list), int(math.sqrt(len(input_list)*13)), 13, bias=True)
trainer = BackpropTrainer(net, DS)

In [102]:
for i in range(5):
    print(trainer.train())

2.21799186291
0.0790515421529
0.043259822245
0.0401598986241
0.0395024913388


In [ ]:
#trainer.trainEpochs(epochs=5)

In [ ]:
#trainer.trainUntilConvergence()

In [103]:
for data in training_data:
    prob = list(net.activate(data['input']))
    data['predicted_class'] = class_names[prob.index(max(prob))]

In [104]:
count = 0
for data in training_data:
    if data['predicted_class'] != data['class']:
        count += 1
print(count/len(training_data))

0.6076923076923076


In [105]:
training_data_fusion_matrix = [[0 for i in range(13)] for i in range(13)]

for data in training_data:
    training_data_fusion_matrix[class_names.index(data['predicted_class'])][class_names.index(data['class'])] += 1
training_data_fusion_matrix = pandas.DataFrame(training_data_fusion_matrix)
training_data_fusion_matrix.columns = class_names #actual_class
training_data_fusion_matrix.index = class_names #predicted_class
training_data_fusion_matrix

,fashion,technology,sports,health,charity,arts,food,other,entertainment,photography,education,travel,business
fashion,89,2,2,6,2,4,3,7,1,0,5,5,0
technology,62,158,73,36,58,78,46,107,80,20,97,38,71
sports,0,0,26,1,1,0,0,1,0,0,0,0,0
health,9,3,33,115,14,6,17,13,13,3,9,5,1
charity,2,2,0,6,49,1,6,3,10,0,5,2,1
arts,2,1,1,2,2,38,0,5,9,0,5,0,6
food,4,4,5,1,2,4,101,7,5,0,3,2,1
other,1,0,0,0,0,0,0,5,1,0,0,0,1
entertainment,0,1,1,1,2,6,2,2,15,0,1,2,0
photography,9,3,5,4,16,14,2,9,14,173,8,10,1


In [106]:
for data in testing_data:
    prob = list(net.activate(data['input']))
    data['predicted_class'] = class_names[prob.index(max(prob))]

In [107]:
count = 0
for data in testing_data:
    if data['predicted_class'] != data['class']:
        count += 1
print(count/len(testing_data))

0.8271650531536047


In [108]:
testing_data_fusion_matrix = [[0 for i in range(13)] for i in range(13)]
class_names = []
for class_name in iter(keyword_tfidf_dict):
    class_names.append(class_name)
for data in testing_data:
    testing_data_fusion_matrix[class_names.index(data['predicted_class'])][class_names.index(data['class'])] += 1
testing_data_fusion_matrix = pandas.DataFrame(testing_data_fusion_matrix)
testing_data_fusion_matrix.columns = class_names #actual_class
testing_data_fusion_matrix.index = class_names #predicted_class
testing_data_fusion_matrix

,fashion,technology,sports,health,charity,arts,food,other,entertainment,photography,education,travel,business
fashion,35,44,22,28,21,180,12,47,71,2,426,20,24
technology,41,1305,287,127,298,2138,284,960,952,19,8815,336,1331
sports,0,8,88,2,1,6,0,1,14,0,65,0,3
health,5,45,119,347,61,315,98,83,134,3,873,50,76
charity,2,28,10,19,153,103,8,33,32,1,425,22,41
arts,3,23,8,4,18,702,22,29,78,3,494,14,70
food,1,42,5,10,7,112,394,45,120,0,237,40,13
other,0,1,4,0,3,10,1,14,5,0,31,1,3
entertainment,0,21,8,8,6,81,18,35,76,1,216,36,23
photography,5,53,26,8,47,392,23,57,111,232,884,96,30
